# POP MOC(sigma 2) for 0.1-degree
**Input Data:** Monthly POP output timeseries files  
**Output Data:** Monthly mean AMOC sigma 2 timeseries  
**Description:** Computes MOC(sigma 2) offline from POP history files using simple xhistogram binning.  
**Date:** February 2023  
**Creator:** Steve Yeager (https://github.com/sgyeager/POP_MOC/blob/main/notebooks/pop_MOCsigma2_0.1deg.ipynb)  
**Updated:** Fred Castruccio and Teagan King, February 2023  
**Note:** To use the MOCutils, a user will need to clone the POP_MOC repository (https://github.com/sgyeager/POP_MOC) and install MOCutils by going to the POP_MOC directory and running `pip install -e . --user`.  
It is also important to request enough memory for this notebook on a casper batch node; suggested allocation is 25GB.

In [1]:
%load_ext watermark
%load_ext autoreload
# %autoreload 2

import cftime
import copy
import dask
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from MOCutils import popmoc
import numpy as np  
import os
import pop_tools
import time
import xarray as xr 
from xhistogram.xarray import histogram

%watermark -iv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
matplotlib: 3.6.3
xarray    : 2023.1.0
dask      : 2023.4.0
json      : 2.0.9
cftime    : 1.6.2
numpy     : 1.23.5
MOCutils  : 0.1
pop_tools : 2023.3.0.post1+dirty
sys       : 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]



In [2]:
from dask.distributed import wait
dask.__version__

'2022.11.0'

In [3]:
pop_tools.__version__

'2023.3.0.post1+dirty'

In [4]:
# Close out Dask Cluster and release workers:
# cluster.close()
# client.close()

In [2]:
# TODO: optimize dask resources

def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=4,
        memory='30GiB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=4:mem=30GB', 
        account='ncgd0011',
        walltime='01:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        #"distributed.scheduler.worker-saturation": "1.2",
        #'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.adapt(minimum_jobs=1, maximum_jobs=30)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.34:46781,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dcherian/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

# Get the required variables 

In [5]:
CHUNKS_LON = 300

# fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002/ocn/proc/tseries/month_1/'
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003/ocn/proc/tseries/month_1/'
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP45/HR/b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'
# fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP26/HR/b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'

# fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.VVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.VVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
# fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
dsV = xr.open_dataset(fin, chunks={'time':1,'nlon':CHUNKS_LON})  # TODO: try chunking by nlat
dsV = time_set_midmonth(dsV,'time')

# fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.WVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.WVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
# fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
dsW = xr.open_dataset(fin, chunks={'time':1,'nlon':CHUNKS_LON})
dsW = time_set_midmonth(dsW,'time')

# fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.TEMP.200601-210012.nc'
fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.TEMP.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.TEMP.200601-210012.nc'
# fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.TEMP.200601-210012.nc'
dsT = xr.open_dataset(fin, chunks={'time':1,'nlon':CHUNKS_LON})
dsT = time_set_midmonth(dsT,'time')

# fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.SALT.200601-210012.nc'
fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.SALT.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.SALT.200601-210012.nc'
# fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.SALT.200601-210012.nc'
dsS = xr.open_dataset(fin, chunks={'time':1,'nlon':CHUNKS_LON})
dsS = time_set_midmonth(dsS,'time')

# fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.UVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.UVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.UVEL.200601-210012.nc'
# fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.UVEL.200601-210012.nc'
dsU = xr.open_dataset(fin, chunks={'time':1,'nlon':CHUNKS_LON})
dsU = time_set_midmonth(dsU,'time')

fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

In [6]:
u_e_all = dsU['UVEL']
u_e_all = u_e_all.where(u_e_all<1.e30,0)
v_e_all = dsV['VVEL']
v_e_all = v_e_all.where(v_e_all<1.e30,0)

# Get model T & S
salt_all = dsS['SALT']
temp_all = dsT['TEMP']

tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

### MOC Region Mask

In [7]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [8]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

In [9]:
%%time
dz = ds_grid['dz'].persist() / 100.
kmt = ds_grid['KMT'].fillna(0).persist() 
print('got dz and kmt')
# Slow step (~12 mins)
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)
print('got dzt dzu')

got dz and kmt
got dzt dzu
CPU times: user 10.4 s, sys: 5.45 s, total: 15.9 s
Wall time: 15.9 s


In [10]:
# Compute sigma-2 field from POP model output
refz = 2000
refdep = xr.DataArray(refz)

# Grid Metrics
dxu = ds_grid['DXU'].reset_coords(drop=True)
dyu = ds_grid['DYU'].reset_coords(drop=True)
dxt = ds_grid['DXT'].reset_coords(drop=True)
dyt = ds_grid['DYT'].reset_coords(drop=True)

# Loop over time slices and compute MOC 

In [14]:
# ystart=[2006,2010,2020,2030,2040,2050,2060,2070,2080,2090,2100]
# yend=[2009,2019,2029,2039,2049,2059,2069,2079,2089,2099,2100]

ystart=[2050,2060,2070,2080,2090,2100]
yend=[2059,2069,2079,2089,2099,2100]

In [13]:
ystart=[2006]
yend=[2100]

In [18]:
# compat="override" avoids equality checking of ULAT, ULONG, TLAT, TLONG
# could be avoided by reading the data differently.
# we do this hear, so that Xarray doesn't repeatedly do those equality checks 
# at many steps in the next cell
merged_all = xr.merge([salt_all, temp_all, u_e_all, v_e_all], compat="override")
merged_all

<xarray.Dataset>
Dimensions:  (z_t: 62, nlat: 2400, nlon: 3600, time: 1140)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.625e+05 5.875e+05
    ULONG    (nlat, nlon) float64 dask.array<chunksize=(2400, 300), meta=np.ndarray>
    ULAT     (nlat, nlon) float64 dask.array<chunksize=(2400, 300), meta=np.ndarray>
    TLONG    (nlat, nlon) float64 dask.array<chunksize=(2400, 300), meta=np.ndarray>
    TLAT     (nlat, nlon) float64 dask.array<chunksize=(2400, 300), meta=np.ndarray>
  * time     (time) object 2006-01-15 00:00:00 ... 2100-12-15 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    SALT     (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 62, 2400, 300), meta=np.ndarray>
    TEMP     (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 62, 2400, 300), meta=np.ndarray>
    UVEL     (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 62, 2400, 300), meta=np.ndarray>
    VVEL     (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 62, 2400, 300), meta=np.ndarray>
Attributes:
    long_name:     Salinity
    units:         gram/kilogram
    grid_loc:      3111
    cell_methods:  time: mean

In [ ]:
%%time
from xarray.tests import raise_if_dask_computes
from dask.distributed import performance_report

with raise_if_dask_computes():
    with performance_report(filename="moc-0.1.html"):
        for n in range(len(ystart)):
            print("started MOC calculations for {}-{} at {}".format(ystart[n], yend[n], time.ctime()))
            merged = merged_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))

            # Sigma2 on model TLAT, TLONG
            sigma2_T = pop_tools.eos(salt=merged.SALT,temp=merged.TEMP,depth=refdep) - 1000
            sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

            # Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
            sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

            # Grid-oriented Volume FLuxes:
            u_e = (merged.UVEL*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
            v_e = (merged.VVEL*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})

            # Convert u_e,v_e to C-grid fluxes
            print('compute to cgrid')
            u = 0.5*(u_e+u_e.shift(nlat=1))
            v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

            # Volume fluxes in density-space. 
            print('compute volume fluxes in density space')
            iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
            iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
            iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
            iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

            # Compute Vertical Volume Flux from horizontal flux convergence
            wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
            wflux = wflux.assign_coords({'TLAT':tlat,'TLONG':tlon})

            # Compute MOC
            print('compute MOC')
            MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
            MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
            #MOC = dask.optimize(MOC)[0]

            # add vflux at southern boundary of Atlantic domain
            print("add vflux")
            tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
            tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
            moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
            moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))

            # This assignment is a bad idea, I don't know why.
            # MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s
            MOC = xr.concat([
                MOC.isel(transport_reg=0), 
                MOC[{'transport_reg':1}] + moc_s
            ], dim="transport_reg")

            # Save to netcdf
            print('groupby')
            MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
            dsout = MOCann.to_dataset()

        #outdir = os.path.dirname(fin)
        #fout = os.path.split(fin)[-1].split('.')[:-3]
        #fout.append('MOCsig')
        #fout.append('{:04d}01-{:04d}12'.format(dsout.time.values[0],dsout.time.values[-1]))
        #fout.append('nc')
        #fout = '.'.join(fout)
        #fout = os.path.join(outdir,fout)
        #dsout.to_netcdf(fout,unlimited_dims='time')
        #print("wrote {} at {}".format(fout, time.ctime()))
        dsout.load(scheduler=client)

started MOC calculations for 2006-2100 at Fri Apr 14 20:08:45 2023
compute to cgrid
compute volume fluxes in density space
compute MOC
add vflux
groupby


/glade/u/home/dcherian/miniconda3/envs/ipogs/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 301.89 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


## Deepak's notes

1. `MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s` this assignment is majorly slow. I replaced it with a `concat` statement
2. it's taking a while for the scheduler to go through the task graph
3. raise chunksize along `nlon` to 300, 1/3 the number of tasks now

4. DXU, DYU are float64, but UVEL etc are float64, this means `iso_uflux`, `iso_vflux`, `wflux` are all promoted to `float64`. Is that a good idea?